# **TON CRM Analytics - Data Cleaning**

**Impotando bibliotecas necessárias**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 50)

## Data Cleaning

**Importando CSVs para o Jupyter Notebook**

In [2]:
cases = pd.read_csv('data/cases.csv', date_parser = ['date_ref']) # Cases
creds = pd.read_csv('data/creds.csv', date_parser = ['cred_date']) # Credenciamento

### Limpeza do dataset `cases`

In [3]:
cases.head(5)

,Unnamed: 0,accountid,date_ref,channelid,waitingtime,missed,pesquisa_de_satisfa_o__c,assunto,Id
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,0013j00002z0CeEAAU,2020-07-31,2.0,15.0,False,NaN,Aplicativo:Dúvidas funcionalidades App:Primeir...,0013j00002z0CeEAAU


**O dataset `cases` não está no melhor formato para a análise.**

Para a limpeza dos dados serão necessários os seguintes passos:

1. Dropar coluna `Unnamed: 0`;
2. Renomear colunas para facilitar o manuseio da base;
3. Remover entradas onde todos os valores são nulos;
4. Parsear a coluna `date_ref` para o formato datetime;
5. Organiar entradas por `data_ref`, em ordem crescente;
6. Checar se as colunas `accountid` (PK) e `Id` são redundantes e, caso forem, eliminar `Id`;
7. Estudar valores em `channelid` e `wairingtime`, e, caso necessário, transformá-los em `int`;
8. Checar se existem entradas duplicada e, em caso positivo, remover redundancias;
9. Estudar coluna `assunto` e entender a melhor maneira de guardar informação de maneira estruturada e aplicar;

#### Renomeando colunas

In [4]:
# Renomeando colunas:
cases.columns = ['unnamed',
                 'account_id', 
                 'date_ref',
                 'channel_id',
                 'waiting_time',
                 'missed', 
                 'pesq_satisfacao', 
                 'assunto', 
                 'id']

cases.head(3)

,unnamed,account_id,date_ref,channel_id,waiting_time,missed,pesq_satisfacao,assunto,id
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Dropando `unnamed`

In [5]:
# Drop "Unnamed: 0"
cases.duplicated('unnamed').sum() # Confirmando se foram criados valores duplicados 
                                     # no índice na criação do dataset
cases = cases.drop('unnamed', axis = 1)

#### Removendo nulos

Removendo entradas (linhas) com _todos_ os valores nulos:

In [6]:
cases['account_id'].isna().sum() # PK Cases faltantes

49500

In [7]:
# Removendo nulos
ccases = cases.dropna(how = 'all', axis = 0) # ccases = cleaned cases
ccases.shape

# Reportando alterações
print("Qtd entradas dataset original:        ", len(cases), 
      "\nQtd entradas dataset sem NaNs:         ", len(ccases), 
      "\nQtd entradas descartadas após dropna:  ", (len(cases)-len(ccases)))

Qtd entradas dataset original:         126989 
Qtd entradas dataset sem NaNs:          77489 
Qtd entradas descartadas após dropna:   49500


In [8]:
# Checando valores nulos remanescentes
ccases.isna().sum()

account_id             0
date_ref               0
channel_id             0
waiting_time           0
missed                 0
pesq_satisfacao    65904
assunto                0
id                     0
dtype: int64

#### Investigando `accountid`

**Dúvida: `accountid` atende os requisitos para ser Primary Key do dataset?**

Conforme dicionário dos dados, a coluna `accountid` deveria ser a chave primária (Primary Key, PK) da tabela, para atender a esse requisito, ela deve ser única em cada uma das linhas.

No entando, parece razoavel pensar que no espaço temporal compreendido neste dataset, um mesmo cliente (cada um com um account id único, pode ter realizado varios chamados na central de relacionamento (CR).

Vamos verificar:

In [9]:
# Checando duplicidade de 'accountid' no dataset
cases.duplicated('account_id').value_counts()

True     95559
False    31430
dtype: int64

**31430 valores _False_** indica que, conforme intuido anteriormente, varias contas chamaram à CR mais de uma vez. Portanto, essa não pode ser a chave primária da tabela.

Como não há uma outra que possa ocupar essa posição, manteremos o index do dataset como chave primária, em que cada linha representa um chamado junto à empresa.

#### Ordenando dataset a partir de `date_ref`

In [10]:
# Ordenando dataset
ccases = ccases.sort_values('date_ref')

#### Checando redundancia
Verificando se informação das colunas `accountid` (PK) e `Id` é a mesma.

In [11]:
# Checando se existem valores diferentes em account_id e id
(ccases['account_id'] != ccases['id']).sum()

0

In [12]:
# Dropando coluna redundante 'id'
ccases = ccases.drop('id', axis = 1)
# ccases.head(3)

#### Revisando `channel_id`
Estudar valores em `channelid` e parsear para `int`, se for o caso.

In [13]:
ccases['channel_id'].value_counts() # Apenas um valor encontrado; De acordo com o dicionário de dados este canal
                                    # são chamados recebidos via telefone. A coluna pode ser descartada, pois
                                    # a análise agora é especifica para esse canal.

2.0    77489
Name: channel_id, dtype: int64

In [14]:
# Dropando coluna "channel_id"
ccases.drop('channel_id', axis = 1, inplace = True)
# ccases.head(3)

#### Checando entradas duplicadas

In [15]:
ccases.duplicated().sum() # Não existem entradas duplicadas.

0

In [16]:
ccases = ccases.reset_index(drop = True)
ccases.head(5)

,account_id,date_ref,waiting_time,missed,pesq_satisfacao,assunto
0,0011L00002ZbpnlQAB,2020-02-23,19.0,False,Enviado,Produto:mPOS:Dúvidas mpos
1,0011L00002dbBg5QAE,2020-02-25,15.0,False,Enviado,Aplicativo:Problema:
2,0011L00002WdgbcQAB,2020-02-26,15.0,False,Enviado,Aplicativo:Dúvidas funcionalidades App:Redefin...
3,0011L00002WdJgjQAF,2020-02-26,13.0,False,Enviado,Produto:mPOS:Problema POS - revertido
4,0011L00002We7cjQAB,2020-02-26,72.0,False,Enviado,Aplicativo::


In [17]:
# ccases.tail(5)

#### Parseando `date_ref` 
Mudando dados da colunas para formato datetime

In [18]:
# Formatando data type da coluna "date_ref"
ccases['date_ref'] = pd.to_datetime(ccases['date_ref'])
# ccases.info()

#### Feature engineering dados temporais
Transformando coluna `waiting_time` (medido em segundos) para dtype `int` e `timedelta` (para operações com data e hora)

In [19]:
ccases['waiting_time'] = ccases['waiting_time'].astype(int)

In [20]:
ccases['waiting_time'] = ccases['waiting_time'].astype(int)
ccases['waiting_timedelta'] = pd.to_timedelta(ccases['waiting_time'], unit = 'S')

In [21]:
def explode_date(df, date_column, language_ptbr = True):
    '''Generates year, month, day and weekday from a single date column.
        
    :::::::
    
    df:            DataFrame containing the date column to be exploded.
    date_column:   Date column (Series) to be used as base for the transformation.
    language_ptbr: If True, parse weekday names to Brazilian Portuguese. 
                      False, weekdays will be parsed to English.
    
    :::::::

    '''
    df[date_column] = pd.to_datetime(df[date_column])
    df['year'] = df[date_column].dt.year 
    df['month'] = df[date_column].dt.month
    df['day'] = df[date_column].dt.day
    df['weekday'] = df[date_column].dt.weekday
    
    if language_ptbr:
        df['weekday_str'] = df['weekday'].map({
                                    0: 'Seg',
                                    1: 'Ter',
                                    2: 'Qua',
                                    3: 'Qui',
                                    4: 'Sex',
                                    5: 'Sab',
                                    6: 'Dom'
                            })
    else:
        df['weekday_str'] = df[date_column].dt.day_name().str[0:3]

    return df

In [22]:
ccases = explode_date(ccases, 'date_ref', language_ptbr = True)
ccases.head(3)

,account_id,date_ref,waiting_time,missed,pesq_satisfacao,assunto,waiting_timedelta,year,month,day,weekday,weekday_str
0,0011L00002ZbpnlQAB,2020-02-23,19,False,Enviado,Produto:mPOS:Dúvidas mpos,00:00:19,2020,2,23,6,Dom
1,0011L00002dbBg5QAE,2020-02-25,15,False,Enviado,Aplicativo:Problema:,00:00:15,2020,2,25,1,Ter
2,0011L00002WdgbcQAB,2020-02-26,15,False,Enviado,Aplicativo:Dúvidas funcionalidades App:Redefin...,00:00:15,2020,2,26,2,Qua


#### Estrutundo dados em `assunto`

In [23]:
# Separando os tópicos encadeados e guardando em uma lista
ccases['assunto'] = ccases['assunto'].str.split(':')

In [24]:
# Criando colunas especificas para cada um dos tópicos
ccases[['node_1','node_2', 'node_3']] = pd.DataFrame(ccases['assunto'].tolist(), index = ccases.index)

# Preenchendo colunas nulas com NaNs
ccases[['node_1', 'node_2', 'node_3']] = ccases[['node_1', 'node_2', 'node_3']].replace('', np.nan)
ccases.head(3)

,account_id,date_ref,waiting_time,missed,pesq_satisfacao,assunto,waiting_timedelta,year,month,day,weekday,weekday_str,node_1,node_2,node_3
0,0011L00002ZbpnlQAB,2020-02-23,19,False,Enviado,"[Produto, mPOS, Dúvidas mpos]",00:00:19,2020,2,23,6,Dom,Produto,mPOS,Dúvidas mpos
1,0011L00002dbBg5QAE,2020-02-25,15,False,Enviado,"[Aplicativo, Problema, ]",00:00:15,2020,2,25,1,Ter,Aplicativo,Problema,NaN
2,0011L00002WdgbcQAB,2020-02-26,15,False,Enviado,"[Aplicativo, Dúvidas funcionalidades App, Rede...",00:00:15,2020,2,26,2,Qua,Aplicativo,Dúvidas funcionalidades App,Redefinição de senha


### Limpeza do dataset `creds`.

In [25]:
creds.head(3)

,Unnamed: 0,cred_date,shipping_address_city,shipping_address_state,max_machine,accountid
0,0,2020-04-18,Feira de Santana,BA,T1,NaN
1,1,2020-10-16,Bacuri,MA,T1,NaN
2,2,2020-09-01,Bernardo Sayão,TO,T1,NaN


#### Renomeando colunas

In [26]:
creds.columns = ['unnamed', 'cred_date', 'ship_city', 'ship_state', 'max_machine', 'account_id']
creds.head(3)

,unnamed,cred_date,ship_city,ship_state,max_machine,account_id
0,0,2020-04-18,Feira de Santana,BA,T1,NaN
1,1,2020-10-16,Bacuri,MA,T1,NaN
2,2,2020-09-01,Bernardo Sayão,TO,T1,NaN


#### Dropando `unnamed`

In [27]:
ccreds = creds.drop('unnamed', axis = 1)
ccreds.head(3)

,cred_date,ship_city,ship_state,max_machine,account_id
0,2020-04-18,Feira de Santana,BA,T1,NaN
1,2020-10-16,Bacuri,MA,T1,NaN
2,2020-09-01,Bernardo Sayão,TO,T1,NaN


#### Ordenando dataset a partir de `cred_date`

In [28]:
ccreds = ccreds.sort_values(by = 'cred_date').reset_index(drop = True)

#### Parseando `cred_date`

In [29]:
explode_date(ccreds, 'cred_date', language_ptbr = True)
ccreds.head(3)

,cred_date,ship_city,ship_state,max_machine,account_id,year,month,day,weekday,weekday_str
0,2019-07-11,São Paulo,SP,NONE,NaN,2019,7,11,3,Qui
1,2019-07-23,Mogi das Cruzes,SP,NONE,NaN,2019,7,23,1,Ter
2,2019-07-23,São Luís,MA,NONE,NaN,2019,7,23,1,Ter


#### Checando entradas duplicadas

In [30]:
print(f'{ccreds.duplicated().sum()} \
        entradas devem ser removidas') # Existem entradas duplicadas que 
                                       # precisam ser removidas

60832         entradas devem ser removidas


In [31]:
# Removendo entradas duplicadas
ccreds = ccreds.drop_duplicates()
creds.head(3)

,unnamed,cred_date,ship_city,ship_state,max_machine,account_id
0,0,2020-04-18,Feira de Santana,BA,T1,NaN
1,1,2020-10-16,Bacuri,MA,T1,NaN
2,2,2020-09-01,Bernardo Sayão,TO,T1,NaN


Algumas entradas que não foram classificadas como duplicadas não possuem `account_id`.

É uma situação interessante. Conforme o dicionário dos dados, esse é um dataset de clientes credenciados, que conta, inclusive, com data de credenciamento. Como vimos, muitos valores estavam duplicados. É possível, portanto, que existam mais linhas duplicadas, mas  essa informação, sem que se conheça o valor `account_id`, é mera especulação. E especulação e uma análise _data-driven_ são o exato oposto uma da outra...

Em princícpio considerei que essas entradas pudessem representar um primeiro contato com a central de relacionamento, um _lead_. Mas a maioria das linhas possui uma máquina de pagamentos associada. Se há uma maquina associada, então, em meu entendimento limitado,, haveria de exister um `account_id`.

Em uma situação ideal, gostaria de conversar com o dono deste dataset para poder entendê-la um pouco melhor e descobrir se é um erro de importação que apaga alguns valores de `account_id` ou erro de algum outro tipo.

Por ora, vou salvar esses dados em um outro dataset, específico com entradas sem `account_id` e dropar essas linhas do dataset principal `ccred`.

#### Salvando entradas sem `account_id` em um novo dataset, `ccreds_naid`

In [32]:
ccreds_naid = ccreds[ccreds['account_id'].isna()]
ccreds_naid.head()

,cred_date,ship_city,ship_state,max_machine,account_id,year,month,day,weekday,weekday_str
0,2019-07-11,São Paulo,SP,NONE,NaN,2019,7,11,3,Qui
1,2019-07-23,Mogi das Cruzes,SP,NONE,NaN,2019,7,23,1,Ter
2,2019-07-23,São Luís,MA,NONE,NaN,2019,7,23,1,Ter
3,2019-07-23,Recife,PE,T1,NaN,2019,7,23,1,Ter
4,2019-07-23,Teresina,PI,NONE,NaN,2019,7,23,1,Ter


In [43]:
ccreds_naid.shape[0] # Total de entradas sem account_id

34728

**Verificando entradas sem uma máquina de pagamento associada (`max_machine` = NONE)**

In [41]:
ccreds_naid[ccreds_naid['max_machine'] == 'NONE'].shape[0] # De 34.728 entradas sem registo de
                                                           # account_id, apenas 752 não possuem
                                                           # uma máquina associada

752

#### Removendo nulos

In [34]:
ccreds = ccreds.dropna(subset = ['account_id'])
ccreds.head(3)

,cred_date,ship_city,ship_state,max_machine,account_id,year,month,day,weekday,weekday_str
10,2019-07-24,Curitiba,PR,T1,0011L00002WdC77QAF,2019,7,24,2,Qua
12,2019-07-24,Recife,PE,T1,0011L00002WdB4gQAF,2019,7,24,2,Qua
17,2019-07-24,Betim,MG,T1,0011L00002Wd777QAB,2019,7,24,2,Qua


In [47]:
# Checando se todos os 'account_id's restantes do dataset possuem valores únicos 
ccreds.shape[0] == ccreds['account_id'].nunique()

True

---

### Exportando dados limpos

In [51]:
ccases.to_csv('data/cases_cleaned.csv', index = False)
ccreds.to_csv('data/creds_cleaned.csv', index = False)
ccreds_naid.to_csv('data/creds_noids.csv', index = False)

**O próximo passo é a análise dos dados propriamente dita.**

Para melhor organização, será realizada em um novo Notebook.

In [50]:
ccases.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77489 entries, 0 to 77488
Data columns (total 15 columns):
account_id           77489 non-null object
date_ref             77489 non-null datetime64[ns]
waiting_time         77489 non-null int64
missed               77489 non-null object
pesq_satisfacao      11585 non-null object
assunto              77489 non-null object
waiting_timedelta    77489 non-null timedelta64[ns]
year                 77489 non-null int64
month                77489 non-null int64
day                  77489 non-null int64
weekday              77489 non-null int64
weekday_str          77489 non-null object
node_1               77383 non-null object
node_2               75770 non-null object
node_3               65564 non-null object
dtypes: datetime64[ns](1), int64(5), object(8), timedelta64[ns](1)
memory usage: 8.9+ MB


# TO INCLUDE IN THE ANALYSIS

In [35]:
# # Contando quantidade de tópicos por node
# print('Qtd. tópicos no node_1:  ', ccases['node_1'].nunique(dropna = False))
# print('Qtd. tópicos no node_2:  ', ccases['node_2'].nunique(dropna = False))
# print('Qtd. tópicos no node_3: ', ccases['node_3'].nunique(dropna = False), '\n')

# print('Qtd. valores faltantes no node_1:   ', ccases['node_1'].isna().sum(), '<-- What are those?')
# print('Qtd. valores faltantes no node_2:  ', ccases['node_2'].isna().sum())
# print('Qtd. valores faltantes no node_3: ', ccases['node_3'].isna().sum())

# # Transformando tópicos em sets para avaliar se são subsets uns dos outros
# topics_node_1 = set(ccases['node_1'].value_counts(dropna = False).index.tolist())
# topics_node_2 = set(ccases['node_2'].value_counts(dropna = False).index.tolist())
# topics_node_3 = set(ccases['node_3'].value_counts(dropna = False).index.tolist())

In [36]:
# # Checando se sets são subsets uns dos outros
# topics_node_2.issubset(topics_node_1), \
# topics_node_3.issubset(topics_node_2), \
# topics_node_3.issubset(topics_node_1)   

In [37]:
# ccases[ccases['node_1'].isna()].head(10)